In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import pickle
from programa_geovars import *
from preprocessing import *

/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
with open('X_train.pkl', 'rb') as f:
    Xtr = pickle.load(f)
with open('X_test.pkl', 'rb') as f:
    Xte = pickle.load(f)

In [2]:
totaltr = pd.read_csv('TOTAL_TRAIN.csv')

/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (54,155) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
totaltr.CODIGO_POSTAL.unique()

array([28822.,    nan])

In [3]:
arm = pd.read_csv('variables_cod_postal_lon_lat.csv')

In [4]:
from scipy.spatial import distance

def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    return closest_index

In [5]:
points = [[l for l in totaltr[['lon','lat']].iloc[ii]] for ii in tqdm(range(totaltr.shape[0]))]

100%|██████████| 103230/103230 [02:45<00:00, 623.03it/s]


In [6]:
comparing_points = [[l for l in arm[['lon','lat']].iloc[ii]] for ii in tqdm(range(arm.shape[0]))]

100%|██████████| 266/266 [00:00<00:00, 632.56it/s]


In [7]:
closest_nodes = [closest_node(point, comparing_points) for point in tqdm(points)]

100%|██████████| 103230/103230 [00:26<00:00, 3874.54it/s]


In [9]:
len(np.unique(closest_nodes))

25

In [10]:
#[arm[col].iloc[which_points[i]]  for i in range(len(which_points))]

In [11]:
for col in tqdm(arm.columns):
    totaltr[col] = [arm[col].iloc[closest_nodes[i]]  for i in range(len(closest_nodes))]


100%|██████████| 56/56 [01:49<00:00,  1.96s/it]


In [12]:
totaltr.to_csv('new_total_train.csv', header=True, index=False)

In [16]:
totaltr.ruido.unique()

array(['Baja', 'ruido_No_Registrado', 'Alta', 'Moderada'], dtype=object)

In [2]:

X, y, encoder = preprocess_data('new_total_train.csv', scale=True, scaler = 'std', process_cat = True, y_name='CLASE', sample_trials=None)

/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (54) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
  0%|          | 0/103230 [00:00<?, ?it/s]/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
  0%|          | 493/103230 [00:00<00:41, 2452.90it/s]

Valores unicos de CADASTRAL--- ['2' '3' '4' '8' '5' '6' '7' '9' '1' 'C' 'B' 'A' 7 6 3 4 5 8 9 2 1 nan]


100%|██████████| 103230/103230 [01:00<00:00, 1700.58it/s]


En momento 2 el shape es de (103230, 195)
Las columnas que tienen dtype object son Index(['ruido', 'cluster', 'TIPOLOGIA_LITERAL', 'CODIGO_POSTAL',
       'medianaEdad'],
      dtype='object')
En momento 3 el shape es de (103230, 334)
Imputando valores con Random Forest
Iteration: 0
Iteration: 1
Iteration: 2
En momento 4 el shape es de (103230, 334)
En momento 5 el shape es de (103230, 344)
En momento 6 el shape es de (103230, 344)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=100)

In [20]:
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight='balanced', verbose=1)

In [21]:
%time
rf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 29.6 µs


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   10.7s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

In [22]:
rf.score(X_test, y_test)

[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 370 tasks      | elapsed:    0.3s
[Parallel(n_jobs=40)]: Done 500 out of 500 | elapsed:    0.4s finished


0.912431385211495

In [23]:
from sklearn.metrics import f1_score

In [24]:
f1_score(y_test, rf.predict(X_test), average='macro')

[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 370 tasks      | elapsed:    0.3s
[Parallel(n_jobs=40)]: Done 500 out of 500 | elapsed:    0.3s finished


0.4757293980455434

In [25]:
f1_score(y_train, rf.predict(X_train), average='macro')

[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.3s
[Parallel(n_jobs=40)]: Done 370 tasks      | elapsed:    0.8s
[Parallel(n_jobs=40)]: Done 500 out of 500 | elapsed:    1.0s finished


1.0